<a href="https://colab.research.google.com/github/FredyOrjuela90/Be-star-Bayesian-Inference-Analysis/blob/main/Notebooks/observables_OldBeAtlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This code reads the Fullsed_v5, Source_v4, and Temperatures_v2 files generated by HDUST.
# It uses the 'read_everything.py' script to load all models based on a grid of values.
#
# Carciofi, A. C., & Bjorkman, J. E. (2006). Non-LTE Monte Carlo radiative transfer. I. THE THERMAL PROPERTIES
# OF KEPLERIAN DISKS AROUND CLASSICAL Be STARS.
# The Astrophysical Journal, 639(2), 1081–1094. https://doi.org/10.1086/499485
#
# ================================
# 1. Mount Google Drive
# ================================
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ================================
# 2. PATH CONFIGURATION
#    (EDIT ONLY THESE LINES)
# ================================
import os
import glob

# Project base path in YOUR Drive:
# Replace "BeAtlas_OLD" with your actual folder name
BASE_DIR = "/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas"   # ⬅️ AJUST
BASE_DIR_DOS = "/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas"

# Subfolders containing the grid data
FULLSED_DIR = os.path.join(BASE_DIR_DOS, "fullsed_v5")   # ⬅️ ADJUST the name if your folder is named differently
TEMPS_DIR   = os.path.join(BASE_DIR_DOS, "temperatures_v2")     # ⬅️ idem

# Output folder for figures and .inp files
OUTPUT_DIR  = os.path.join(BASE_DIR_DOS, "Output")    # ⬅️ You can choose another one

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("BASE_DIR  :", BASE_DIR,  "->", os.path.exists(BASE_DIR))
print("FULLSED   :", FULLSED_DIR, "->", os.path.exists(FULLSED_DIR))
print("SOURCE    :", SOURCE_DIR,  "->", os.path.exists(SOURCE_DIR))
print("TEMPS     :", TEMPS_DIR,   "->", os.path.exists(TEMPS_DIR))
print("OUTPUT    :", OUTPUT_DIR,  "->", os.path.exists(OUTPUT_DIR))

print("\nEjemplos de archivos encontrados:")
print("  fullsed:", len(glob.glob(os.path.join(FULLSED_DIR, "*"))))
print("  source :", len(glob.glob(os.path.join(SOURCE_DIR, "*"))))
print("  temps  :", len(glob.glob(os.path.join(TEMPS_DIR, "*"))))


BASE_DIR  : /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas -> True
FULLSED   : /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/fullsed_v5 -> True
SOURCE    : /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/source_v4 -> True
TEMPS     : /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2 -> True
OUTPUT    : /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/Output -> True

Ejemplos de archivos encontrados:
  fullsed: 1133
  source : 390
  temps  : 1507


In [ ]:
# ================================
# 3. Install Dependencies
# ================================
!pip install -q numpy scipy matplotlib pyhdust
# Install libraries; pyhdust handles HDUST simulations
# pyhdust requires Python >= 3.6, compatible with current Colab
# https://pyhdust.readthedocs.io/


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.5/28.5 MB 29.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# ================================
# 4. Add project folder to sys.path
#    to import your .py modules
# ================================
import sys

# Assuming observables_OldBeAtlas.py and read_everything.py
# are located directly in BASE_DIR
sys.path.append(BASE_DIR)

print("sys.path incluye BASE_DIR?", BASE_DIR in sys.path)

# If we change the read_everything file or update paths
import importlib, read_everything
importlib.reload(read_everything)  # In case you modify the .py


sys.path incluye BASE_DIR? True


<module 'read_everything' from '/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/read_everything.py'>

In [ ]:
# ================================
# 5. (Optional but VERY USEFUL)
#    Patch read_everything to use Drive paths
# ================================
import inspect
import types
import read_everything

# --- Check original behavior (for inspection only) ---
print("read_everything.read_everything definido en:", inspect.getsourcefile(read_everything))
print("Nombre de la función:", read_everything.read_everything.__name__)

# ⚠️ IMPORTANT:
# We are NOT touching the internal code here yet.
# First, we test if it can be called and see what paths it returns.


read_everything.read_everything definido en: /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/read_everything.py
Nombre de la función: read_everything


In [ ]:
# ================================
# Modify (Patch) the external script "observables_OldBeAtlas.py"
# The original script uses a hardcoded relative path ("./../../") which may fail in Colab.
# This block finds that line and replaces it with our absolute OUTPUT_DIR.
# ================================


# Read the script content
path_script = os.path.join(BASE_DIR, "observables_OldBeAtlas.py")

with open(path_script, "r") as f:
    txt = f.read()

# Define the old string (legacy relative path) and the new string (absolute path)
old = 'dest_folder = "./../../"'
new = f'dest_folder = r"{OUTPUT_DIR}/"'


# Perform the replacement only if the old string is found
if old in txt:
    txt = txt.replace(old, new)
    with open(path_script, "w") as f:
        f.write(txt)
    print("✔ dest_folder parcheado para usar:", OUTPUT_DIR)
else:
    print("⚠ No se encontró la línea dest_folder = \"./../../\" (quizá ya está cambiada).")


⚠ No se encontró la línea dest_folder = "./../../" (quizá ya está cambiada).


In [ ]:
# ================================
# 6. EXECUTE DATA LOADING & VERIFICATION
#    Call the main function to load the model grid and print the
#    returned paths/counts to verify everything is correct.
# ================================

import read_everything

try:
    files_fullsed_new, files_source_new, files_temps_new, fullsed_contents, \
        fullsed_path, source_path, temps_path, dist_std = read_everything.read_everything()

    print("✅ read_everything() se ejecutó correctamente.\n")
    print("Rutas que devuelve internamente:")
    print("  fullsed_path:", fullsed_path)
    print("  source_path :", source_path)
    print("  temps_path  :", temps_path)
    print("  dist_std    :", dist_std)
    print("\nTamaños:")
    print("  N fullsed:", len(files_fullsed_new))
    print("  N source :", len(files_source_new))
    print("  N temps  :", len(files_temps_new))
    print("  fullsed_contents (modelos leídos):", len(fullsed_contents))
except Exception as e:
    print("❌ ERROR al llamar read_everything.read_everything():")
    print(e)


Reading the OldBeAtlas files...

# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.10_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 6.35e-10 Msun (Rstar=4.1e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.20_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 9.89e-10 Msun (Rstar=4.4e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.30_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 1.53e-09 Msun (Rstar=4.8e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.40_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 2.28e-

In [ ]:
# ================================
# 7. DIRECT FILE CHECK (DEBUGGING)
#    Use glob to independently verify the files actually present in the directories.
# ================================

files_fullsed_raw = glob.glob(fullsed_path + "*")
files_source_raw  = glob.glob(source_path + "*")
files_temps_raw   = glob.glob(temps_path + "*")

print("Archivos brutos en fullsed_path:", len(files_fullsed_raw))
print("Archivos brutos en source_path :", len(files_source_raw))
print("Archivos brutos en temps_path  :", len(files_temps_raw))

print("Ejemplos fullsed:", [os.path.basename(f) for f in files_fullsed_raw[:5]])


Archivos brutos en fullsed_path: 1133
Archivos brutos en source_path : 390
Archivos brutos en temps_path  : 1507
Ejemplos fullsed: ['fullsed_mod625_PLn3.5_sig0.28_h072_Rd050.0_Be_M20.00_ob1.40_H0.30_Z0.014_bE_Ell.sed2', 'fullsed_mod32_PLn3.0_sig0.12_h072_Rd050.0_Be_M07.70_ob1.40_H0.30_Z0.014_bE_Ell.sed2', 'fullsed_mod222_PLn4.0_sig0.68_h072_Rd050.0_Be_M09.60_ob1.10_H0.30_Z0.014_bE_Ell.sed2', 'fullsed_mod105_PLn3.5_sig0.05_h072_Rd050.0_Be_M06.40_ob1.45_H0.30_Z0.014_bE_Ell.sed2', 'fullsed_mod453_PLn2.0_sig0.05_h072_Rd050.0_Be_M04.80_ob1.30_H0.30_Z0.014_bE_Ell.sed2']


In [ ]:

import numpy as np

# Compatibility patch for legacy code.
# Since we are running on a newer version (Python 3+) and the original code was for version 2,
# we alias np.product to np.prod if it is missing.
if not hasattr(np, "product"):
    np.product = np.prod


In [ ]:
# ================================
# INSTALL ADDITIONAL UTILITIES
# wget: Tool for downloading files.
# xmltodict: Library to parse XML files as Python dictionaries.
# ================================
!pip install wget xmltodict


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=24950fbf4c9e8e727beec8a89a9c2ec4a93b4c1f4ea19accda419d788d6e2632
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [ ]:
# ================================
# 8. RUN OBSERVABLES SCRIPT
#    Change working directory to BASE_DIR to ensure internal paths work,
#    then execute the main processing script.
#    The 'operations_on_stars.inp' file is generated.
# ================================

os.chdir(BASE_DIR)
print("Directorio actual:", os.getcwd())

%run observables_OldBeAtlas.py


Directorio actual: /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas
Reading the OldBeAtlas files...

# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.10_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 6.35e-10 Msun (Rstar=4.1e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.20_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 9.89e-10 Msun (Rstar=4.4e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_Be_M04.20_ob1.30_H0.30_Z0.014_bE_Ell30_avg.temp completely read!
# Disk mass is 1.53e-09 Msun (Rstar=4.8e+00 Rsun)
# XDR /content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/OldBeAtlas/temperatures_v2/mod442_PLn2.0_sig0.02_h072_Rd050.0_B

/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/observables_OldBeAtlas.py:468: RuntimeWarning: invalid value encountered in sqrt
  return 0.+A/np.sqrt(2.*np.pi*sigma2)*np.exp(-0.5*(x-0.)**2./sigma2)
/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/observables_OldBeAtlas.py:468: RuntimeWarning: overflow encountered in exp
  return 0.+A/np.sqrt(2.*np.pi*sigma2)*np.exp(-0.5*(x-0.)**2./sigma2)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/pyhdust/spectools.py:511: UserWarning: Wrong `lbc` in the lineProf function
  _warn.warn("Wrong `lbc` in the lineProf function")
/content/drive/MyDrive/Be-HDUST/Project_Lband_v3/Study_OldBeAtlas/observables_OldBeAtla

SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING IS WRONG WITH THIS HDUST OUTPUT!
SOMETHING I

/usr/local/lib/python3.12/dist-packages/pyhdust/lrr/__init__.py:1269: RuntimeWarning: divide by zero encountered in scalar divide
  C0 = auxi_flambda[-1]/auxi_lamb[-1]**alpha0
/usr/local/lib/python3.12/dist-packages/pyhdust/lrr/__init__.py:1253: RuntimeWarning: invalid value encountered in scalar multiply
  return C*x**alpha
/usr/local/lib/python3.12/dist-packages/pyhdust/lrr/__init__.py:1359: RuntimeWarning: divide by zero encountered in log10
  return -2.5*np.log10(X)+zp


Se han truncado las últimas 5000 líneas del flujo de salida.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to photon flux.
Writing NaN to 